In [ ]:
import pandas as pd

df = pd.read_csv('AIS_History_Info.csv')

def drop_duplicate(data):
    data.drop('Unnamed: 0', axis=1, inplace=True)
    
    # 중복제거
    data.drop_duplicates(inplace = True)
    
    return data


def header_update(data):
    # header 추출
    index = data.columns.values.tolist()

    # 대문자 변경
    index = list(map(lambda x: x.upper(), index))

    # 공백 -> _, .제거
    index = list(map(lambda x : x.replace(" ", "_").replace(".", ""), index))

    # header 변경
    data.columns = index
    
    return data


def type_update(data):
    # 어떤 항목을 어떤 타입으로 바꿀지 고민
    print(data.info())
    
    return data


def groupby_ship_id(data):
    # ship_id 별 그룹화 하기 위해 고유한 ship_id 추출
    ship_id_list = data['SHIP_ID'].unique()
    ship_id_list
    
    # 사용할지는 모르지만 df_배이름 별로 새로운 데이터프레임 생성
 #    df_name = []
 #    for ship in ship_id_list:
 #        df_name.append('df_' + ship)

 #    for id_name, ship in enumerate(ship_id_list):
 #         globals()[df_name[id_name]] = df[df['SHIP_ID'] == ship]
            
    # ship_id별 작업처리
    for id_name, ship in enumerate(ship_id_list):
        df_new = data[data['SHIP_ID'] == ship]

        # 필요없는 컬럼 제거
        df_new.drop(['SHIP_NAME','IMO','SOG','SOURCE','ETA','NAV_STATUS','NAV_STATUS_CODE','CALL_SIGN','VESSEL_TYPE','VESSEL_TYPE_CODE','VESSEL_TYPE_CARGO','VESSEL_CLASS','LENGTH','WIDTH','FLAG','FLAG_CODE','COG','ROT','HEADING','VESSEL_TYPE_MAIN','VESSEL_TYPE_SUB','MESSAGE_TYPE'], axis=1, inplace=True)

        # 시간 순 정렬
        df_new.sort_values(by=['DT_POS_UTC'])

        # DESTINATIIPN(-1) 생성
        df_new['DESTINATION(-1)'] = 0
        df_new['DESTINATION(-1)'][1:] = df_new['DESTINATION'][:-1]
        df_new['DESTINATION(-1)'][0] = 'missing'

        # DESTINATION_KEY 생성
        df_new['DESTINATION_KEY'] = 0
        for i in df_new.index:
            if i == 0:
                df_new['DESTINATION_KEY'][i] = 1
            else:
                if df_new['DESTINATION(-1)'][i-1] == 'missing':
                    df_new['DESTINATION_KEY'][i] = df_new['DESTINATION_KEY'][i-1]

                elif df_new['DESTINATION(-1)'][i] == df_new['DESTINATION(-1)'][i-1]:
                    df_new['DESTINATION_KEY'][i] = df_new['DESTINATION_KEY'][i-1]

                else:
                    df_new['DESTINATION_KEY'][i] = df_new['DESTINATION_KEY'][i-1] + 1
        
    # 이후 처리 과정에 따라 업데이트 예정